# Sentiment

Visualize sentiment analysis. This notebook relies on sentiment scores per tweet stored in the directory `data/sentiment/pattern` . These sentiment scores can be generated with the script `scripts/sentiment-pattern-text.py` .

In [ ]:
import datetime
import os
import re
import statistics
import sys
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from IPython.display import clear_output

In [ ]:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [ ]:
DATASOURCE = "twitter"
DATADIRECTORYALL = f"../data/{DATASOURCE}/sentiment/"
DATADIRECTORYTEXT = f"../data/{DATASOURCE}/text/"
SENTIMENT = "sentiment"
COUNT = "count"
DATA = "data"
LABEL = "label"
HIGHLIGHT = "highlight"
HIGHLIGHTLABEL = "highlightlabel"
TEXT ="text"
IDSTR = "id_str"
DATE = "date"

In [ ]:
def squeal(text=None):
    clear_output(wait=True)
    if not text is None: print(text)

## Read hourly sentiment analysis by pattern's sentiment analysis

In [ ]:
DEFAULTFILEPATTERN = "2.*z"

def getSentimentPerHour(dataDirectory,filePattern=DEFAULTFILEPATTERN):
    fileList = sorted(os.listdir(dataDirectory))
    sentimentPerHour = {}
    for inFileName in fileList:
        if re.search(filePattern,inFileName):
            if re.search(r"-00.out.gz",inFileName): squeal(inFileName)
            try: df = pd.read_csv(dataDirectory+inFileName,compression="gzip",header=None)
            except: continue
            sentiment = sum(df[1])/len(df)
            hour = inFileName[0:11]
            sentimentPerHour[hour] = { SENTIMENT:sentiment, COUNT:len(df) }
    sentimentPerHour = {key:sentimentPerHour[key] for key in sorted(sentimentPerHour.keys())}
    return(sentimentPerHour)

## Convert hourly analysis to day scores

In [ ]:
def makeSentimentPerDay(sentimentPerHour):
    sentimentPerDay = {}
    for hour in sentimentPerHour:
        day = re.sub("..$","12",hour)
        if not day in sentimentPerDay: sentimentPerDay[day] = {SENTIMENT:0,COUNT:0}
        sentimentPerDay[day][SENTIMENT] += sentimentPerHour[hour][SENTIMENT]*sentimentPerHour[hour][COUNT]
        sentimentPerDay[day][COUNT] += sentimentPerHour[hour][COUNT]
    for day in sentimentPerDay:
        sentimentPerDay[day][SENTIMENT] /= sentimentPerDay[day][COUNT]
    return(sentimentPerDay)

## Visualize

In [ ]:
DATEFORMATHOUR = "%Y%m%d-%H"
DATEFORMATMONTH = "%-d/%-m"
DATEFORMATHRSMINS = "%H:%M"
DEFAULTTITLE = "Polarity scores of Dutch tweets over time"
PLOTFILENAME = "sentiment-all.png"
ANNOTATE = "annotate"

def stringToDate(dateString):
    return(datetime.datetime.strptime(dateString,DATEFORMATHOUR))

def list_skip(list,skip_factor=0):
    return([list[i] for i in range(0,len(list)) if (i)%(skip_factor+1) == 0])

def visualizeSentiment(dataSources,title=DEFAULTTITLE,dateFormat=DATEFORMATMONTH,skip_factor=0,outFileName=PLOTFILENAME, tuple_data_list=[]):
    font = {"size":16}
    matplotlib.rc("font",**font)
    #fig,ax = plt.subplots(figsize=(12,6))
    fig,ax = plt.subplots(figsize=(16, 9))
    #plt.ylim(-0.14,0.16)
    ax.xaxis.set_major_formatter(mdates.DateFormatter(dateFormat))
    for i in range(0,len(dataSources)):
        data = dataSources[i][DATA]
        label = dataSources[i][LABEL]
        lineData = ax.plot_date(list_skip([stringToDate(key) for key in data if data[key][COUNT] > 1],skip_factor=skip_factor),
                                list_skip([data[key][SENTIMENT] for key in data if data[key][COUNT] > 1],skip_factor=skip_factor),
                                xdate=True,fmt="-",label=label)
        if HIGHLIGHT in dataSources[i]:
            highlight = dataSources[i][HIGHLIGHT]
            color = lineData[-1].get_color()
            if not HIGHLIGHTLABEL in dataSources[i]:
                ax.plot_date([stringToDate(key) for key in highlight],
                             [data[key][SENTIMENT] for key in highlight],\
                             fmt="o",color=color)
            else:
                highlightlabel = dataSources[i][HIGHLIGHTLABEL]
                ax.plot_date([stringToDate(key) for key in highlight],
                             [data[key][SENTIMENT] for key in highlight],\
                             fmt="o",color=color,label=highlightlabel)
        if ANNOTATE in dataSources[i]:
            for date1,date2,text in dataSources[i][ANNOTATE]:
                plt.annotate(text,(stringToDate(date2),data[date1][SENTIMENT]),color=color)
    plt.title(title)
    plt.legend(framealpha=0.2, bbox_to_anchor=(0,1), loc="upper left")
    # plt.legend(framealpha=0.2)
    for tuple_data in tuple_data_list:
        plt.text(datetime.datetime.strptime(tuple_data[0], DATEFORMATHOUR), tuple_data[1], tuple_data[2], fontsize=12)
    plt.savefig(outFileName)
    plt.show()
    return(ax)

## Process data obtained from all Dutch tweets

In [ ]:
press_conferences = ["20200301-12", "20200309-12", "20200312-12", "20200315-12", "20200317-12", "20200319-12", "20200323-12", "20200331-12",
                     "20200407-12", "20200415-12", "20200421-12", "20200429-12",
                     "20200506-12", "20200513-12", "20200519-12", "20200527-12",\
                     "20200603-12", "20200624-12",
                     "20200722-12",
                     "20200806-12", "20200818-12",
                     "20200901-12", "20200918-12", "20200928-12",
                     "20201013-12",
                     "20201103-12", "20201117-12",
                     "20201208-12", "20201214-12",
                     "20210112-12", "20210120-12",
                     "20210202-12", "20210223-12",
                     "20210308-12", "20210323-12",
                    ]
highlight = list(press_conferences)

In [ ]:
annotate = [("20200220-12","20200221-12","(1)"),\
            ("20200302-12","20200303-12","(2)"),\
            ("20200312-12","20200313-12","(3)"),\
            ("20200322-12","20200323-12","(4)"),\
            ("20200330-12","20200331-12","(5)"),\
            ("20200413-12","20200414-12","(6)"),\
            ("20200421-12","20200422-12","(7)"),\
            ("20200428-12","20200429-12","(8)"),\
            ("20200430-12","20200501-12","(9)"),\
            ("20200511-12","20200512-12","(10)"),\
            ("20200526-12","20200527-12","(11)"),\
            ("20200601-12","20200603-12","(12)"),\
            ("20200608-12","20200609-12","(13)"),\
            ("20200622-12","20200623-12","(14)"),\
            ("20200708-12","20200709-12","(15)"),
            ("20200922-12","20200923-12","(16)"),
            ("20201006-12","20201007-12","(17)"),
            ("20201016-12","20201017-12","(18)"),
            ("20201029-12","20201030-12","(19)"),
            ("20201214-12","20201216-12","(20)"),
            ("20201225-12","20201227-12","(21)"),
            ("20210101-12","20210103-12","(22)"),
            ("20210124-12","20210126-12","(23)"),
            ("20210216-12","20210218-12","(24)"),
           ]

In [ ]:
sentimentPerHour = getSentimentPerHour(DATADIRECTORYALL)
sentimentPerDay = makeSentimentPerDay(sentimentPerHour)

In [ ]:
dummy = visualizeSentiment([{DATA:{date:sentimentPerDay[date] for date in sentimentPerDay 
                                                              if date >= "20200201"}, LABEL:"per day",\
                             HIGHLIGHT:highlight,HIGHLIGHTLABEL:"press conference",
                             ANNOTATE:[]}],\
                           title="Daily average sentiment scores of tweets written in Dutch",
                           tuple_data_list = [
                                              ["20200314-12", 0.059, "12-03-2020\nWork at home"],
                                              ["20200602-12", 0.053, "01-06-2020\nBLM demonstration Dam"],
                                              ["20201007-12", 0.058, "06-10-2020\nEddie Van Halen dies"],
                                              ["20201216-12", 0.0635, "14-12-2020\nLockdown"],
                                              ["20201208-12", 0.126, "25-12"],
                                              ["20210102-12", 0.1335, "01-01"],
                                              ["20201223-12", 0.054, "25-01-2021\nRiots"],
                                             ])

In [ ]:
dummy = visualizeSentiment([{DATA:sentimentPerDay,LABEL:"per day",\
                             HIGHLIGHT:highlight,HIGHLIGHTLABEL:"press conference",
                             ANNOTATE:annotate}],\
                           title=DEFAULTTITLE+" (all Dutch tweets)")

```
 (1) & 20200220 & terrorist attack Hanau, Germany \\
 (2) & 20200302 & newsitem: Netherlands financially supporting asylum seekers \\
 (3) & 20200312 & announcement of first national COVID-19 measures \\
 (4) & 20200322 & lack of social distancing in nature areas \\
 (5) & 20200330 & news item: EU donates millions to Morocco \\
 (6) & 20200413 & news item: anti-gay violence \\
 (7) & 20200421 & national press conference on COVID-19 \\
 (8) & 20200428 & news item: lack of social distancing at IKEA \\
 (9) & 20200430 & news item: local coalition of CDA and FvD \\
(10) & 20200511 & news item: etnic profiling by tax services \\
(11) & 20200526 & news item: KLM boss receives bonus \\
(12) & 20200601 & lack of social distancing at Amsterdam BLM demonstration \\
(13) & 20200608 & attacks on left-wing politicians \\
(14) & 20200622 & racism in tv programme Veronica Inside \\
(15) & 20200708 & farmers protest \\
(16) & 20200922 & Dutch VIPs involved in protest against measures (#ikdoenietmeermee) \\
(17) & 20201006 & Eddie Van Halen dies \\
(18) & 20201016 & Paris teacher killed \\
(19) & 20201029 & Nice terror attack \\
(20) & 20201214 & Lockdown starts \\
(21) & 20201225 & Christmas \\
(22) & 20210101 & New Year \\
(23) & 20210124 & Curfew riots \\
(24) & 20210216 & Judge deems curfew illegal \\
```

Find dates of sentiment minima with next code block and then look for relevant topics in tweets with these shell commands:
```
gunzip -c ../sentiment/pattern/20200820-* | cut -d, -f2 > ~/tmp/scores
gunzip -c 20200820-* | python3 ../../scripts/get-text.py | grep -v '^text$' | paste -d , ~/tmp/scores - |\
   sort | uniq -c | sort -nr | less
```

In [ ]:
def sortDatesBySentiment(sentimentPerDay,month=""):
    return([(hourString,sentimentPerDay[hourString]) \
             for hourString in sorted(sentimentPerDay.keys(),key=lambda h:sentimentPerDay[h][SENTIMENT]) 
             if re.search(month,hourString)])

sortDatesBySentiment(sentimentPerDay,month="202010")

## Get sentiment on topics

In [ ]:
DATADIRECTORYSENTTOPIC = "../data/sentiment/topic/"

def getSentimentPerHourQuery(dataDirectory,query,filePattern=DEFAULTFILEPATTERN,computeMissing=True,\
                             dataDirectorySentTopic=DATADIRECTORYSENTTOPIC,dataDirectoryText=DATADIRECTORYTEXT,
                             missingPattern=DEFAULTFILEPATTERN):
    fileList = sorted(os.listdir(dataDirectory))
    sentimentPerHour = {}
    for inFileName in fileList:
        if re.search(filePattern,inFileName):
            if re.search("00.out.gz",inFileName): squeal(inFileName)
            if os.path.exists(dataDirectorySentTopic+inFileName):
                dfSent = pd.read_csv(dataDirectorySentTopic+inFileName,header=None)
                sentiment = sum(dfSent[1])/len(dfSent)
                hour = inFileName[0:11]
                sentimentPerHour[hour] = { SENTIMENT:sentiment, COUNT:len(dfSent) }
            elif computeMissing and re.search(missingPattern,inFileName):
                pd.DataFrame([{0:1}]).T.to_csv(dataDirectorySentTopic+inFileName,header=None,index="0")
                try:
                    dfSent = pd.read_csv(dataDirectory+inFileName,header=None,index_col=0, engine="python")
                    dfText = pd.read_csv(dataDirectoryText+inFileName, index_col="id_str", engine="python")
                except:
                    print(f"error reading data file {inFileName}!")
                    sys.exit(1)
                if len(dfSent) != len(dfText):
                    print(f"lengths of files {inFileName} do not match! {len(dfText)} {len(dfSent)}")
                    sys.exit(1)
                matchesText = dfText[dfText[TEXT].str.contains(query,case=False)]
                matchesSent = dfSent[dfSent.index.isin(matchesText.index)]
                if len(matchesSent) > 0:
                    matchesSent.to_csv(dataDirectorySentTopic+inFileName,header=None)
                    sentiment = sum(list(matchesSent[1]))/len(matchesSent)
                    hour = inFileName[0:11]
                    sentimentPerHour[hour] = { SENTIMENT:sentiment, COUNT:len(matchesSent) }
                else:
                    os.unlink(dataDirectorySentTopic+inFileName)
    sentimentPerHour = {key:sentimentPerHour[key] for key in sorted(sentimentPerHour.keys())}
    return(sentimentPerHour)

In [ ]:
COMPUTEMISSING = True

In [ ]:
TOPICQUERY = "corona|covid|huisarts|mondkapje|rivm|blijfthuis|flattenthecurve|houvol"
PANDEMICQUERYBASE = "|".join([TOPICQUERY, r'virus|besmet|ziekenhui|\bic\b|intensive.care|^zorg|vaccin|[^ad]arts|uitbraak|uitbrak|pandemie|ggd|'+
                                      r'mondkapje|quarantaine|\bwho\b|avondklok|variant|verple|sympto|e.golf|mutant|^omt$|umc|hcq|'+
                                      r'hydroxychloroquine|virolo|zkh|oversterfte|patiënt|patient|intensivist|🦠|ivermectin'])
DISTANCEQUERY = "1[.,]5[ -]*m|afstand.*hou|hou.*afstand|anderhalve[ -]*meter"
LOCKDOWNQUERY = "lock.down|lockdown"
VACCINQUERY = "vaccin|ingeënt|ingeent|inent|prik|spuit|bijwerking|-->|💉|pfizer|moderna|astrazeneca|astra|zeneca|novavax|biontech"
TESTQUERY = r'\btest|getest|sneltest|pcr'

PANDEMICQUERY = "|".join([ DISTANCEQUERY, LOCKDOWNQUERY, PANDEMICQUERYBASE, TESTQUERY, VACCINQUERY])


IKQUERY = r'\b(ik|mij|mijn|me|mn|m\'n|zelf|mezelf|mijzelf|i)\b'
HAPPYQUERY = r'\b(geluk|gelukkig|gelukkige|blij|happy)\b'
LONELYQUERY = r'eenza|alleen.*voel|voel.*alleen|lonely|loneli'
SWEARQUERY = (r'shit|fuck|klote|kanker|kut|lul|dom|reet|gvd|strot|waanzin|nep|rotzooi|idio|schaamtelo|fake|strot|'+
              r'zeik|stom|onbeschoft|oprot|bekrompen|ongelo|bah|kwalijk|onbekwa|achterlijk|jat|schand|triest|puinho|'+
              r'immore|kleptocratie|nepotism|absurd|misselijkma|\bbek\b|\brot\b|\bkk\b')

In [ ]:
QUERYTOPIC = "corona|covid|mondkapje|rivm|blijfthuis|houvol|huisarts|flattenthecurve"

sentimentPerHourTopic = getSentimentPerHourQuery(DATADIRECTORYALL,QUERYTOPIC,filePattern="^202",\
                                                 dataDirectorySentTopic=f"../data/{DATASOURCE}/sentiment-topic/",\
                                                 dataDirectoryText=DATADIRECTORYTEXT,computeMissing=COMPUTEMISSING)

In [ ]:
QUERYTOPIC = "mondkapje"

sentimentPerHourMondkapje = getSentimentPerHourQuery(DATADIRECTORYALL,QUERYTOPIC,filePattern="^202",\
                                                     dataDirectorySentTopic=f"../data/{DATASOURCE}/sentiment-mondkapje/",\
                                                     dataDirectoryText=DATADIRECTORYTEXT,computeMissing=COMPUTEMISSING,
                                                     missingPattern="^20210[2-9]")

In [ ]:
QUERYTOPIC = "1[.,]5[ -]*m|afstand.*hou|hou.*afstand|anderhalve[ -]*meter"

sentimentPerHourDistance = getSentimentPerHourQuery(DATADIRECTORYALL,QUERYTOPIC,filePattern="^202",\
                                                    dataDirectorySentTopic=f"../data/{DATASOURCE}/sentiment-distance/",\
                                                    dataDirectoryText=DATADIRECTORYTEXT,computeMissing=COMPUTEMISSING,
                                                    missingPattern="^20210[2-9]")

In [ ]:
QUERYTOPIC = "lockdown|lock.down"

sentimentPerHourLockdown = getSentimentPerHourQuery(DATADIRECTORYALL, QUERYTOPIC, filePattern="^202",
                                                    dataDirectorySentTopic=f"../data/{DATASOURCE}/sentiment-lockdown/",
                                                    dataDirectoryText=DATADIRECTORYTEXT, computeMissing=COMPUTEMISSING,
                                                    missingPattern="^20210[2-9]")

In [ ]:
QUERYTOPIC = "vaccin"

sentimentPerHourVaccin = getSentimentPerHourQuery(DATADIRECTORYALL, QUERYTOPIC,filePattern="2020*",\
                                                  dataDirectorySentTopic=f"../data/{DATASOURCE}/sentiment-vaccin/",\
                                                  dataDirectoryText=DATADIRECTORYTEXT, computeMissing=COMPUTEMISSING,
                                                  missingPattern="^20210[2-9]")

In [ ]:
QUERYTOPIC = "avondklok"
DIRTOPIC = f"../data/{DATASOURCE}/sentiment-avondklok/"

if not os.path.isdir(DIRTOPIC):
    os.mkdir(DIRTOPIC)
sentimentPerHourCurfew = getSentimentPerHourQuery(DATADIRECTORYALL, 
                                                  QUERYTOPIC,
                                                  filePattern="^202",
                                                  dataDirectorySentTopic=DIRTOPIC,
                                                  dataDirectoryText=DATADIRECTORYTEXT,
                                                  computeMissing=COMPUTEMISSING,
                                                  missingPattern="^20210[2-9]")

In [ ]:
QUERYTOPIC = ('shit|fuck|klote|kut|lul|dom|reet|gvd|strot|waanzin|nep|rotzooi|idio|schaamtelo|fake|strot|'+
    'zeik|stom|onbeschoft|oprot|bekrompen|ongelo|bah|kwalijk|onbekwa|achterlijk|jat|schand|triest|puinho|'+
    'immore|kleptocratie|nepotism|absurd|\sbek\s|\srot\s|\skk\s')
DIRTOPIC = f"../data/{DATASOURCE}/sentiment-swear/"

if not os.path.isdir(DIRTOPIC):
    os.mkdir(DIRTOPIC)
sentimentPerHourSwear = getSentimentPerHourQuery(DATADIRECTORYALL, 
                                                 QUERYTOPIC,
                                                 filePattern="^202",
                                                 dataDirectorySentTopic=DIRTOPIC,
                                                 dataDirectoryText=DATADIRECTORYTEXT,
                                                 computeMissing=COMPUTEMISSING,
                                                 missingPattern="^20210[2-9]")

In [ ]:
QUERYTOPIC = r'\btest|getest|sneltest|pcr'
DIRTOPIC = f"../data/{DATASOURCE}/testing/"

if not os.path.isdir(DIRTOPIC):
    os.mkdir(DIRTOPIC)
sentimentPerHourTest = getSentimentPerHourQuery(DATADIRECTORYALL, 
                                                QUERYTOPIC,
                                                filePattern="^202",
                                                dataDirectorySentTopic=DIRTOPIC,
                                                dataDirectoryText=DATADIRECTORYTEXT,
                                                computeMissing=COMPUTEMISSING,
                                                missingPattern="^202")

In [ ]:
QUERYTOPIC = PANDEMICQUERY
DIRTOPIC = f"../data/{DATASOURCE}/sentiment-pandemic/"

if not os.path.isdir(DIRTOPIC):
    os.mkdir(DIRTOPIC)
sentimentPerHourPandemic = getSentimentPerHourQuery(DATADIRECTORYALL, 
                                                    QUERYTOPIC,
                                                    filePattern="^202",
                                                    dataDirectorySentTopic=DIRTOPIC,
                                                    dataDirectoryText=DATADIRECTORYTEXT,
                                                    computeMissing=COMPUTEMISSING,
                                                    missingPattern="^202")

In [ ]:
sentimentPerDayMondkapje = makeSentimentPerDay(sentimentPerHourMondkapje)
sentimentPerDayDistance = makeSentimentPerDay(sentimentPerHourDistance)
sentimentPerDayTopic = makeSentimentPerDay(sentimentPerHourTopic)
sentimentPerDayLockdown = makeSentimentPerDay(sentimentPerHourLockdown)
sentimentPerDayVaccin = makeSentimentPerDay(sentimentPerHourVaccin)
sentimentPerDayCurfew = makeSentimentPerDay(sentimentPerHourCurfew)
sentimentPerDaySwear = makeSentimentPerDay(sentimentPerHourSwear)
sentimentPerDayPandemic = makeSentimentPerDay(sentimentPerHourPandemic)

In [ ]:
FIRSTCOVIDDAY = "20200312-12"

sentimentPerDayMondkapje = {key:sentimentPerDayMondkapje[key] for key in sentimentPerDayMondkapje.keys() if key >= FIRSTCOVIDDAY}
sentimentPerDayDistance = {key:sentimentPerDayDistance[key] for key in sentimentPerDayDistance.keys() if key >= FIRSTCOVIDDAY}
sentimentPerDayLockdown = {key:sentimentPerDayLockdown[key] for key in sentimentPerDayLockdown.keys() if key >= FIRSTCOVIDDAY}
sentimentPerDayVaccin = {key:sentimentPerDayVaccin[key] for key in sentimentPerDayVaccin.keys() if key >= FIRSTCOVIDDAY}
sentimentPerDayCurfew = {key:sentimentPerDayCurfew[key] for key in sentimentPerDayCurfew.keys() if key >= FIRSTCOVIDDAY}
sentimentPerDaySwear = {key:sentimentPerDaySwear[key] for key in sentimentPerDaySwear.keys() if key >= FIRSTCOVIDDAY}

In [ ]:
(len(sentimentPerDayMondkapje), len(sentimentPerDayDistance), len(sentimentPerDayLockdown), len(sentimentPerDayVaccin),
 len(sentimentPerDayCurfew), len(sentimentPerDaySwear), len(sentimentPerDayPandemic), len(sentimentPerDayTopic), len(sentimentPerDay))

## General graphs

In [ ]:
annotateTopic = [("20200215-12","20200216-12","(A)"),
                 ("20200223-12","20200224-12","(B)"),
                 ("20200308-12","20200309-12","(C)"),
                 ("20200322-12","20200323-12","(D)"),
                 ("20200411-12","20200412-12","(E)"),
                 ("20200420-12","20200421-12","(F)"),
                 ("20200510-12","20200511-12","(G)"),
                 ("20200519-12","20200520-12","(H)"),
                 ("20200601-12","20200602-12","(I)"),
                 ("20200621-12","20200622-12","(J)"),
                 ("20200712-12","20200713-12","(K)"),
                 ("20200716-12","20200717-12","(L)"),
                 ("20200813-12","20200814-12","(M)"),
                 ("20200901-12","20200902-12","(N)"),
                 ("20210124-12","20210125-12","(O)"),
                ]

In [ ]:
PLOTFILENAME = "sentiment-all.png"
STARTDATE = "20200215-00"

dummy = visualizeSentiment([{DATA:{date:sentimentPerDay[date] for date in sentimentPerDay if date >= STARTDATE}, 
                             LABEL:"all tweets",
                             HIGHLIGHT:highlight,
                             ANNOTATE:annotate},
                            {DATA:{date:sentimentPerDayPandemic[date] for date in sentimentPerDayPandemic if date >= STARTDATE},
                             LABEL:"pandemic tweets",
                             HIGHLIGHT:highlight,
                             HIGHLIGHTLABEL:"press conference",
                             ANNOTATE:annotateTopic}],
                           title=DEFAULTTITLE+" (all Dutch tweets vs topic tweets)",outFileName=PLOTFILENAME)

```
(A) & 20200215 & COVID-19 death on Dutch cruiseship \\
(B) & 20200223 & Italy outbreak \\
(C) & 20200308 & news item: COVID-19 spreading in Europe \\
(D) & 20200322 & news item: lack of social distancing in nature areas \\
(E) & 20200411 & news item: Netherlands sent facemasks to China \\
(F) & 20200420 & anti-climate protest on Twitter \\
(G) & 20200510 & warnings about staying alert \\
(H) & 20200519 & relaxation of COVID-19 measures announced \\
(I) & 20200601 & lack of social distancing at Amsterdam BLM demonstration \\
(J) & 20200621 & The Hague demonstration cancelled for lack of social distancing \\
(K) & 20200712 & news item: facemask related murder in France \\
(L) & 20200716 & discussion about initial health care worker safety \\
(M) & 20200813 & coalition leaves parliament to avoid vote on health care worker salaries \\
(N) & 20200902 & new photos showing lack of social distancing wedding Grapperhaus \\
(O) & 20210124 & Curfew riots \\
```

In [ ]:
#sortDatesBySentiment(sentimentPerDay,month="202103")
sortDatesBySentiment(sentimentPerDayPandemic,month="202101")

In [ ]:
def get_extreme_tweets(date, topic, negative=False):
    df = {}
    for hour in range(0,24):
        hour = str(hour).zfill(2)
        file = f"{date}-{hour}.out.gz"
        tweets = pd.read_csv(DATADIRECTORYTEXT+file, index_col="id_str")
        sentiment = pd.read_csv(DATADIRECTORYALL+file, header=None, index_col=0)
        sentiment = sentiment.rename(columns={1:"sentiment"}, index={0:"id_str"})
        tweets_sent = tweets.join(sentiment)[["text","sentiment"]]
        if topic != "":
            tweets_sent = tweets_sent[tweets_sent["text"].str.contains(topic)]
        if len(df) == 0: 
            df = tweets_sent
        else:
            df = pd.concat([df,tweets_sent])
    groups = df.groupby(["text","sentiment"]).groups
    if negative:
        return([(len(groups[group]),group) for group in sorted(groups, key=lambda group:len(groups[group]), reverse=True) if group[1] < 0])
    else:
        return([(len(groups[group]),group) for group in sorted(groups, key=lambda group:len(groups[group]), reverse=True) if group[1] > 0])

In [ ]:
QUERYTOPIC = "corona|covid|mondkapje|rivm|blijfthuis|houvol|huisarts|flattenthecurve"
ALLTWEETSTOPIC = ""

get_extreme_tweets("20210212", QUERYTOPIC, negative=True)[:20]

## Graph for EMNLP paper

In [ ]:
annotatePaper = [("20200312-12","20200313-12","(A)"),
                 ("20200519-12","20200520-12","(B)")]

dummy = visualizeSentiment([{DATA:sentimentPerDay,LABEL:"all tweets",
                             HIGHLIGHT:[],HIGHLIGHTLABEL:"",ANNOTATE:[]},
                            {DATA:sentimentPerDayTopic,LABEL:"topic tweets",HIGHLIGHT:[],
                             ANNOTATE:annotatePaper}],
                           title=DEFAULTTITLE+" (all Dutch tweets vs topic tweets)")

In [ ]:
def show_dates_higher_covid_sentiment():
    print([(d,sentimentPerDay[d][SENTIMENT],sentimentPerDayTopic[d][SENTIMENT]) for d in sentimentPerDayTopic \
           if sentimentPerDayTopic[d][SENTIMENT] > sentimentPerDay[d][SENTIMENT]])

show_dates_higher_covid_sentiment()

In [ ]:
[(hourString,sentimentPerDayTopic[hourString]) \
 for hourString in sorted(sentimentPerDayTopic.keys(),key=lambda h:sentimentPerDayTopic[h][SENTIMENT]) if re.search("202010",hourString)]

### Two per-topic graphs

In [ ]:
highlightPaper = ["20200312-12","20200519-12"]
annotatePaper = [("20200312-12","20200313-12","(A)"),
                 ("20200519-12","20200520-12","(B)")]
STARTDATE = "20210101-12"

dummy = visualizeSentiment([
            {DATA:sentimentPerDay,LABEL:"all",HIGHLIGHT:[],HIGHLIGHTLABEL:"",ANNOTATE:[]},
            {DATA:sentimentPerDayPandemic,LABEL:"COVID-19",HIGHLIGHT:highlightPaper,HIGHLIGHTLABEL:"",ANNOTATE:annotatePaper}],
            title=DEFAULTTITLE)
dummy = visualizeSentiment([
            {DATA:{k:sentimentPerDayMondkapje[k] for k in sentimentPerDayMondkapje if k >= STARTDATE},LABEL:"face masks",HIGHLIGHT:"",HIGHLIGHTLABEL:"",ANNOTATE:""},
            {DATA:{k:sentimentPerDayDistance[k] for k in sentimentPerDayDistance if k >= STARTDATE},LABEL:"distancing",HIGHLIGHT:"",HIGHLIGHTLABEL:"",ANNOTATE:""},
            {DATA:{k:sentimentPerDayLockdown[k] for k in sentimentPerDayLockdown if k >= STARTDATE},LABEL:"lockdown",HIGHLIGHT:"",HIGHLIGHTLABEL:"",ANNOTATE:""},
            {DATA:{k:sentimentPerDayVaccin[k] for k in sentimentPerDayVaccin if k >= STARTDATE},LABEL:"vaccin",HIGHLIGHT:"",HIGHLIGHTLABEL:"",ANNOTATE:""},
            {DATA:{k:sentimentPerDayCurfew[k] for k in sentimentPerDayCurfew if k >= STARTDATE},LABEL:"curfew",HIGHLIGHT:"",HIGHLIGHTLABEL:"",ANNOTATE:""},
            {DATA:{k:sentimentPerDaySwear[k] for k in sentimentPerDaySwear if k >= STARTDATE},LABEL:"swear",HIGHLIGHT:"",HIGHLIGHTLABEL:"",ANNOTATE:""},
            {DATA:{k:sentimentPerDayPandemic[k] for k in sentimentPerDayPandemic if k >= STARTDATE},LABEL:"pandemic",HIGHLIGHT:"",HIGHLIGHTLABEL:"",ANNOTATE:""},
            ],
            title=DEFAULTTITLE+" (per day)")

In [ ]:
def movingAverage(sentiment_per_day,size):
    sentiment_per_day_items = list(sentiment_per_day.items())
    new_sentiment_per_day_items = []
    for i in range(0,len(sentiment_per_day_items)):
        new_sentiment_per_day_items.append((sentiment_per_day_items[i][0],{SENTIMENT:0, COUNT:0}))
        for j in range(i-size+1+round(0.5*size),i+1+round(0.5*size)):
            if j >= 0 and j < len(sentiment_per_day):
                new_sentiment_per_day_items[-1][1][SENTIMENT] += sentiment_per_day_items[j][1][SENTIMENT]*sentiment_per_day_items[j][1][COUNT]
                new_sentiment_per_day_items[-1][1][COUNT] += sentiment_per_day_items[j][1][COUNT]
        new_sentiment_per_day_items[-1][1][SENTIMENT] /= new_sentiment_per_day_items[-1][1][COUNT]
    return(dict(new_sentiment_per_day_items))

In [ ]:
def movingMedian(sentiment_per_day, size):
    sentiment_per_day_items = list(sentiment_per_day.items())
    new_sentiment_per_day_items = []
    for i in range(0,len(sentiment_per_day_items)):
        new_sentiment_per_day_items.append([sentiment_per_day_items[i][0], []])
        for j in range(i-size,i+1+size):
            if j >= 0 and j < len(sentiment_per_day):
                new_sentiment_per_day_items[-1][1].append(sentiment_per_day_items[j][1][SENTIMENT])
        new_sentiment_per_day_items[-1][1] = { SENTIMENT:statistics.median(new_sentiment_per_day_items[-1][1]), 
                                               COUNT: sentiment_per_day_items[i][1][COUNT] }
    return(dict(new_sentiment_per_day_items))

In [ ]:
def pretty_number(number):
    digits = str(number)
    pretty_digits = ""
    for i in range(0,len(digits)):
        if i > 0 and i % 3 == 0: pretty_digits = ","+pretty_digits
        pretty_digits = digits[-1-i]+pretty_digits
    return(pretty_digits)

In [ ]:
NBROFDAYS = 60
EXTRASMOOTHDAYS = 30
MEDIANCONTEXT = 14

#mondkapjeData = movingAverage(movingAverage(sentimentPerDayMondkapje, NBROFDAYS), EXTRASMOOTHDAYS)
#distanceData = movingAverage(movingAverage(sentimentPerDayDistance, NBROFDAYS), EXTRASMOOTHDAYS)
#lockdownData = movingAverage(movingAverage(sentimentPerDayLockdown, NBROFDAYS), EXTRASMOOTHDAYS)
#vaccinData = movingAverage(movingAverage(sentimentPerDayVaccin, NBROFDAYS), EXTRASMOOTHDAYS)
#curfewData = movingAverage(movingAverage(sentimentPerDayCurfew, NBROFDAYS), EXTRASMOOTHDAYS)
#swearData = movingAverage(movingAverage(sentimentPerDaySwear, NBROFDAYS), EXTRASMOOTHDAYS)

mondkapjeData = movingMedian(sentimentPerDayMondkapje, MEDIANCONTEXT)
distanceData = movingMedian(sentimentPerDayDistance, MEDIANCONTEXT)
lockdownData = movingMedian(sentimentPerDayLockdown, MEDIANCONTEXT)
vaccinData = movingMedian(sentimentPerDayVaccin, MEDIANCONTEXT)
curfewData = movingMedian(sentimentPerDayCurfew, MEDIANCONTEXT)
swearData = movingMedian(sentimentPerDaySwear, MEDIANCONTEXT)
pandemicData = movingMedian(sentimentPerDayPandemic, MEDIANCONTEXT)

mondkapjeDayCounts = np.sum([sentimentPerDayMondkapje[day]["count"] for day in sentimentPerDayMondkapje])
distanceDayCounts = np.sum([sentimentPerDayDistance[day]["count"] for day in sentimentPerDayDistance])
lockdownDayCounts = np.sum([sentimentPerDayLockdown[day]["count"] for day in sentimentPerDayLockdown])
vaccinDayCounts = np.sum([sentimentPerDayVaccin[day]["count"] for day in sentimentPerDayVaccin])
curfewDayCounts = np.sum([sentimentPerDayCurfew[day]["count"] for day in sentimentPerDayCurfew])
swearDayCounts = np.sum([sentimentPerDaySwear[day]["count"] for day in sentimentPerDaySwear])
pandemicDayCounts = np.sum([sentimentPerDayPandemic[day]["count"] for day in sentimentPerDayPandemic])

dummy = visualizeSentiment([
            {DATA:mondkapjeData, LABEL:f"face masks ({pretty_number(mondkapjeDayCounts)})", HIGHLIGHT:"", HIGHLIGHTLABEL:"", ANNOTATE:""},
            {DATA:lockdownData, LABEL:f"lockdown ({pretty_number(lockdownDayCounts)})", HIGHLIGHT:"", HIGHLIGHTLABEL:"", ANNOTATE:""},
            {DATA:distanceData, LABEL:f"distancing ({pretty_number(distanceDayCounts)})", HIGHLIGHT:"", HIGHLIGHTLABEL:"", ANNOTATE:""},
            {DATA:vaccinData, LABEL:f"vaccination ({pretty_number(vaccinDayCounts)})", HIGHLIGHT:"", HIGHLIGHTLABEL:"", ANNOTATE:""},
            {DATA:curfewData, LABEL:f"curfew ({pretty_number(curfewDayCounts)})", HIGHLIGHT:"", HIGHLIGHTLABEL:"", ANNOTATE:""},
            {DATA:swearData, LABEL:f"swear ({pretty_number(swearDayCounts)})", HIGHLIGHT:"", HIGHLIGHTLABEL:"", ANNOTATE:""},
            {DATA:pandemicData, LABEL:f"pandemic ({pretty_number(pandemicDayCounts)})", HIGHLIGHT:"", HIGHLIGHTLABEL:"", ANNOTATE:""},
        ], title=DEFAULTTITLE+f" (median over {1+2*MEDIANCONTEXT} days)",skip_factor=0)

### Two plots in one figure

In [ ]:
PLOTFILENAME = "sentiment-all-split.png"
highlightPaperAll = ["20200312-12"]
annotatePaperAll = [("20200312-12","20200313-12","(A)")]
highlightPaperCovid = ["20200519-12"]
annotatePaperCovid = [("20200519-12","20200520-12","(B)")]

dateFormat = DATEFORMATMONTH
dataSources = [{DATA:sentimentPerDay,LABEL:"all",HIGHLIGHT:highlightPaperAll,HIGHLIGHTLABEL:"",ANNOTATE:annotatePaperAll},
               {DATA:sentimentPerDayPandemic,LABEL:"COVID-19",HIGHLIGHT:highlightPaperCovid,HIGHLIGHTLABEL:"",ANNOTATE:annotatePaperCovid}]
title = DEFAULTTITLE
YMIN = -0.13
YMAX =0.17

if True:
    font = {"size":12}
    matplotlib.rc("font",**font)
    fig,ax = plt.subplots(figsize=(12,6))
    ax = plt.subplot(121)
    plt.ylim(YMIN,YMAX) # was -0.14,0.18
    ax.xaxis.set_major_formatter(mdates.DateFormatter(dateFormat))
    for i in range(0,len(dataSources)):
        data = dataSources[i][DATA]
        label = dataSources[i][LABEL]
        lineData = ax.plot_date([stringToDate(key) for key in data if data[key][COUNT] > 1],\
                     [data[key][SENTIMENT] for key in data if data[key][COUNT] > 1],xdate=True,fmt="-",label=label)
        if HIGHLIGHT in dataSources[i]:
            highlight = dataSources[i][HIGHLIGHT]
            color = lineData[-1].get_color()
            if not HIGHLIGHTLABEL in dataSources[i]:
                ax.plot_date([stringToDate(key) for key in highlight],
                             [data[key][SENTIMENT] for key in highlight],\
                             fmt="o",color="black")
            else:
                highlightlabel = dataSources[i][HIGHLIGHTLABEL]
                ax.plot_date([stringToDate(key) for key in highlight],
                             [data[key][SENTIMENT] for key in highlight],\
                             fmt="o",color="black",label=highlightlabel)
        if ANNOTATE in dataSources[i]:
            for date1,date2,text in dataSources[i][ANNOTATE]:
                plt.annotate(text,(stringToDate(date2),data[date1][SENTIMENT]),color="black")
    plt.title("                               Polarity scores of")
    plt.xlabel("dates from February to October 2020")
    plt.ylabel("polarity score (min: -1.0; max: +1.0)")
    plt.legend(framealpha=0.2)

dataSources = [{DATA:sentimentPerDayDistance,LABEL:"distancing",HIGHLIGHT:highlightPaper,HIGHLIGHTLABEL:"",ANNOTATE:annotatePaper},
               {DATA:sentimentPerDayMondkapje,LABEL:"face masks",HIGHLIGHT:highlightPaper,HIGHLIGHTLABEL:"",ANNOTATE:annotatePaper},
               {DATA:sentimentPerDayLockdown,LABEL:"lockdown",HIGHLIGHT:highlightPaper,HIGHLIGHTLABEL:"",ANNOTATE:annotatePaper},
               {DATA:sentimentPerDayVaccin,LABEL:"vaccin",HIGHLIGHT:highlightPaper,HIGHLIGHTLABEL:"",ANNOTATE:annotatePaper},
               {DATA:sentimentPerDayCurfew,LABEL:"curfew",HIGHLIGHT:highlightPaper,HIGHLIGHTLABEL:"",ANNOTATE:annotatePaper},
               {DATA:sentimentPerDaySwear,LABEL:"swear",HIGHLIGHT:highlightPaper,HIGHLIGHTLABEL:"",ANNOTATE:annotatePaper},
               {DATA:sentimentPerDayPandemic,LABEL:"pandemic",HIGHLIGHT:highlightPaper,HIGHLIGHTLABEL:"",ANNOTATE:annotatePaper},
              ]

if True:
    ax = plt.subplot(122)
    plt.ylim(YMIN,YMAX)
    ax.xaxis.set_major_formatter(mdates.DateFormatter(dateFormat))
    for i in range(0,len(dataSources)):
        data = dataSources[i][DATA]
        label = dataSources[i][LABEL]
        x = [key for key in data if data[key][COUNT] > 1]
        y = [data[key][SENTIMENT] for key in data if data[key][COUNT] > 1]
        lineData = ax.plot_date([stringToDate(key) for key in x],y,xdate=True,fmt="-",label=label)
        plottedData = {x[i]:y[i] for i in range(0,len(x))}
        #if HIGHLIGHT in dataSources[i]:
        #    highlight = dataSources[i][HIGHLIGHT]
        #    color = lineData[-1].get_color()
        #    if not HIGHLIGHTLABEL in dataSources[i]:
        #        ax.plot_date([stringToDate(key) for key in highlight],
        #                     [plottedData[key] for key in highlight],\
        #                     fmt="o",color=color)
        #    else:
        #        highlightlabel = dataSources[i][HIGHLIGHTLABEL]
        #        ax.plot_date([stringToDate(key) for key in highlight],
        #                     [plottedData[key] for key in highlight],\
        #                     fmt="o",color=color,label=highlightlabel)
        #if ANNOTATE in dataSources[i]:
        #    for date1,date2,text in dataSources[i][ANNOTATE]:
        #        plt.annotate(text,(stringToDate(date2),plottedData[date1]),color=color)
    plt.title("Dutch tweets over time                      ")
    plt.legend(framealpha=0.2)
    plt.xlabel("dates from March to October 2020")
    plt.savefig(PLOTFILENAME)
    plt.show()

In [ ]:
def averageSentiment(sentimentPerDay):
    sentiment = 0.0
    count = 0
    for date in sentimentPerDay.keys():
        sentiment += sentimentPerDay[date][SENTIMENT]*sentimentPerDay[date][COUNT]
        count += sentimentPerDay[date][COUNT]
    return(sentiment/count)

print(averageSentiment(sentimentPerDayMondkapje),
      averageSentiment(sentimentPerDayDistance),
      averageSentiment(sentimentPerDayLockdown),
      averageSentiment(sentimentPerDayVaccin),
      averageSentiment(sentimentPerDaySwear),
      averageSentiment(sentimentPerDayPandemic),
     )

Average sentiment scores:
1. Twitter: face masks: 0.037; distancing: 0,064; lockdown: 0.054 (-1 Nov)
2. Nu.nl: face masks: 0.055; distancing: 0.056 (-1 Aug)
3. Reddit: face masks: 0.050; distancing: 0.068 (-1 Aug)

## STOP HERE

## Show positive tweets

In [ ]:
def showTweetTextWithSentiment(dataDir,query,filePattern=DEFAULTFILEPATTERN):
    fileList = sorted(os.listdir(dataDir))
    tweetSentPairs = []
    for inFileName in fileList:
        if re.search(filePattern,inFileName):
            squeal(inFileName)
            if os.path.exists(dataDir+inFileName):
                dfSent = pd.read_csv(dataDir+inFileName,header=None,index_col=0)
                dfText = pd.read_csv(DATADIRECTORYTEXT+inFileName,index_col=IDSTR)
                selectedDict = {idStr:dfText.loc[idStr][TEXT] for idStr in dfText.index if re.search(query,dfText.loc[idStr][TEXT],flags=re.IGNORECASE)}
                for idStr in selectedDict.keys():
                    try:
                        tweetSentPairs.append((dfSent.loc[idStr][1],selectedDict[idStr]))
                    except: pass
    tweetSentPairs = sorted(tweetSentPairs,key=lambda e:e[0],reverse=True)
    return(tweetSentPairs)

tweetSentPairs = showTweetTextWithSentiment(DATADIRECTORYSENTTOPIC,query="coronapocalypse",filePattern="20200316")

In [ ]:
tweetSentPairs[0:10]

In [ ]:
sum([tweetSentPairs[i][0] for i in range(0,len(tweetSentPairs))])/len(tweetSentPairs)

## Visualize sentiment of single press conference day

In [ ]:
def convertDate(dateString1,dateString2,dateString3):
    return(re.sub(dateString1,dateString2,dateString3))

In [ ]:
dummy = visualizeSentiment([{DATA:{k:sentimentPerHour[k] for k in sentimentPerHour.keys() \
                                   if re.search(r"^20200312",k)},\
                             LABEL:"20200312",HIGHLIGHT:["20200312-15"],HIGHLIGHTLABEL:"press conference"},\
                            {DATA:{convertDate("20200311","20200312",k):sentimentPerHour[k] for k in sentimentPerHour.keys() 
                                   if re.search(r"^20200311",k)},LABEL:"20200311"},\
                            {DATA:{convertDate("20200313","20200312",k):sentimentPerHour[k] for k in sentimentPerHour.keys() 
                                   if re.search(r"^20200313",k)},LABEL:"20200313"}],\
                           title="Sentiment scores of Dutch tweets on Thursday 12 March 2020",dateFormat=DATEFORMATHRSMINS)

In [ ]:
dummy = visualizeSentiment([{DATA:{k:sentimentPerHour[k] for k in sentimentPerHour.keys() \
                                   if re.search(r"^20200312",k)},\
                             LABEL:"all tweets",HIGHLIGHT:["20200312-15"],HIGHLIGHTLABEL:"press conference"}],\
                           title="Polarity scores of Dutch tweets on Thursday 12 March 2020",dateFormat=DATEFORMATHRSMINS)

In [ ]:
{DATA:{k:sentimentPerHour[k] for k in sentimentPerHour.keys() if re.search(r"^20200311",k)},LABEL:""}

## Compare day graph with previous years

In [ ]:
DATADIRECTORYALL2019 = "../data/sentiment/2019/"
TEMPERATURESFILE = "../data/temperatures.csv" 

def changeYearTo2020(dataIn):
    dataOut = {}
    for dateString in dataIn:
        dateString2020 = re.sub(r"20[0-9][0-9]","2020",dateString)
        dataOut[dateString2020] = dataIn[dateString]
    return(dataOut)

sentimentPerHour2019 = getSentimentPerHour(DATADIRECTORYALL2019)
sentimentPerDay2019 = makeSentimentPerDay(sentimentPerHour2019)
ax = visualizeSentiment([{DATA:sentimentPerDay,LABEL:"2020 per day",\
                          HIGHLIGHT:highlight,HIGHLIGHTLABEL:"press conference"},\
                         {DATA:changeYearTo2020(sentimentPerDay2019),LABEL:"2019 per day",\
                          HIGHLIGHT:["20200315-12","20200318-12"],HIGHLIGHTLABEL:"terror attack"}],
                         title=DEFAULTTITLE)

## Add daily temperatures to plot

In [ ]:
def readTemperatures():
    temperatures = pd.read_csv(TEMPERATURESFILE,header=None)
    temperatures = { (str(temperatures[0][i])+"-12"):(temperatures[1][i]/10) for i in range(0,len(temperatures))}
    return(temperatures)

temperatures = readTemperatures()

ax2 = ax.twinx()
ax2.set_ylim([-20,40])
dummy = ax2.plot_date([datetime.datetime.strptime(key,DATEFORMATHOUR) for key in temperatures],\
                      [temperatures[key] for key in temperatures],xdate=True,\
                      fmt="--",color="990000",label="temperature")

## Check relation temperature - sentiment

In [ ]:
plt.figure(figsize=(3,3))
plt.gcf().subplots_adjust(left=0.2)
x = [sentimentPerDay[key][SENTIMENT] for key in temperatures]
y = [temperatures[key] for key in temperatures]
plt.scatter(x,y)
plt.show()

In [ ]:
np.corrcoef(x,y)[0][1]

## Check relation sentiment today - yesterday 

In [ ]:
plt.figure(figsize=(3,3))
plt.gcf().subplots_adjust(left=0.2)
x = [sentimentPerDay[key][SENTIMENT] for key in sentimentPerDay]
y = [sentimentPerDay[key][SENTIMENT] for key in sentimentPerDay]
x.pop(-1)
y.pop(0)
plt.scatter(x,y)
plt.show()

In [ ]:
np.corrcoef(x,y)[0][1]

## Sort days by sentiment score

Dates of Dutch government press conferences related to corona crisis:

1. Sunday 1 March 2020: https://www.youtube.com/watch?v=rg9aM97Pu38
2. Monday 9 March 2020: https://www.youtube.com/watch?v=xz-DtYHGdLA
3. Thursday 12 March 2020: https://www.youtube.com/watch?v=0iD1FN6I87Y (start: around 15:15 PM)
4. Sunday 15 March 2020: https://www.youtube.com/watch?v=j94ULn90xg8
5. Tuesday 17 March 2020: https://www.youtube.com/watch?v=KuXj3c1F8WY
6. Thursday 19 March 2020: https://www.youtube.com/watch?v=Yqx0PlSnUFE
7. Monday 23 March 2020: https://www.youtube.com/watch?v=mcpLFX_L9o8
8. Tuesday 31 March 2020: https://www.youtube.com/watch?v=FNuzw3wplow
9. Tuesday 7 April 2020: https://www.youtube.com/watch?v=-j3_mmZcBZU
10. Wednesday 15 April 2020: https://www.youtube.com/watch?v=n1-rSb3j0UM
11. Tuesday 21 April 2020: https://www.youtube.com/watch?v=Yjx7SPtq7Bk
12. Wednesday 29 April 2020: https://www.youtube.com/watch?v=dfhzIGagbOw
13. Wednesday 6 May 2020: https://www.youtube.com/watch?v=rBtDphRcPKA
14. Wednesday 13 May 2020: https://www.youtube.com/watch?v=zISG9KkuAQ0
15. Tuesday 19 May 2020: https://www.youtube.com/watch?v=MlAk_RMTHZU
16. Wednesday 27 May 2020: https://www.youtube.com/watch?v=M0qBpBz_5h8
17. Wednesday 3 June 2020: https://www.youtube.com/watch?v=rsIUKfLjf8k

In [ ]:
{k:round(v[SENTIMENT],4) for k,v in sorted(sentimentPerDay.items(),key=lambda item:item[1][SENTIMENT])}

## Process data from RIVM query

In [ ]:
sentimentPerHour = getSentimentPerHour(DATADIRECTORYRIVM)
sentimentPerDay = makeSentimentPerDay(sentimentPerHour)
visualizeSentiment([{DATA:sentimentPerHour,LABEL:"per hour"},{DATA:sentimentPerDay,LABEL:"per day"}],\
                   title=DEFAULTTITLE+" (RIVM query)")

## Sort days by sentiment score

In [ ]:
{k:v for k,v in sorted(sentimentPerDay.items(),key=lambda item:item[1][SENTIMENT])}

## Examine tweet texts from day with extreme sentiment

In [ ]:
import gzip
import json
import numpy as np
from library import getTweetText

def removeNewlines(text):
    return(re.sub(r"\n",r" ",text))

HOURSPERDAY = 24
DATE = "20200901"
TWEETDIRECTORYRIVM = "../data/rivm/"

tweetTexts = []
for hour in range(0,HOURSPERDAY):
    hour = str(hour).zfill(2)
    inFile = gzip.open(TWEETDIRECTORYRIVM+DATE+"-"+hour+".rivm.gz")
    for line in inFile:
        jsonData = json.loads(line)
        tweetText = removeNewlines(getTweetText(jsonData))
        tweetTexts.append(tweetText)
    inFile.close()
    
pd.DataFrame(tweetTexts)[0].value_counts()[0:20]

## Process sentiment of topic

In [ ]:
def readSentiment(fileName):
    return(pd.read_csv(fileName,index_col="date").T.to_dict())

In [ ]:
DATADIR = "../data/sentiment/"
TOPIC = "corona|covid|flattenthecurve|blijfthuis|rivm|mondkapje|huisarts|houvol"
EXTENSION = ".csv"

# sentimentPerHourTopic = readSentiment(DATADIR+TOPIC+EXTENSION)
sentimentPerHourTopic = getSentiment()

In [ ]:
sentimentPerDayTopic = makeSentimentPerDay(sentimentPerHourTopic)
dummy = visualizeSentiment([{DATA:sentimentPerDay,LABEL:"all",
                            HIGHLIGHT:highlight,HIGHLIGHTLABEL:"press conference"},
                            {DATA:sentimentPerDayTopic,LABEL:"topic",\
                             HIGHLIGHT:highlight,HIGHLIGHTLABEL:"press conference"}],\
                           title=DEFAULTTITLE+" (all Dutch tweets vs topic tweets)")

In [ ]:
for selectedDates in "202002 202003 202004 202005[01]".split():
    nbrOfTweetsAll = sum([sentimentPerDay[date][COUNT] for date in sentimentPerDay if re.search(selectedDates,date)])
    nbrOfTweetsTopic = sum([sentimentPerDayTopic[date][COUNT] for date in sentimentPerDayTopic if re.search(selectedDates,date)])
    print(selectedDates,round(nbrOfTweetsTopic/nbrOfTweetsAll,3))

In [ ]:
sentimentPerDay